In [2]:
import json
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, GlobalAveragePooling1D, Dense
import random  
import tensorflow as tf
import pickle

In [3]:
# Step 1: Load Data
def load_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data['tags']

hozana_intents= load_data('sample_intents.json')
hozana_intents

[{'tag': 'greeting',
  'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'How are you?'],
  'responses': ['Hello', 'Hi', 'Hi there', 'Hello how are you?']},
 {'tag': 'thanks',
  'patterns': ['Thanks', 'Thank you', "That's helpful", 'Thanks for the help'],
  'responses': ['Happy to help!',
   'Any time!',
   'My pleasure',
   "You're most welcome!"]}]

In [4]:
# Step 2: Add New tags, Patterns, and Responses
new_intents = [
    {
        "tag": "Order Coffee",
        "patterns": ["I'd like to order a coffee.","Can I get a coffee, please?","I want to buy some coffee.", "A coffee, please"],
        "responses": ["Sure, what type of coffee would you like?","Of course, which size would you prefer?","Great choice! Would you like anything else with your coffee?"]
    },
    {
        "tag": "Customize Order",
        "patterns": ["I'd like my coffee with milk.","Can you make my coffee extra hot?","I want just a black coffee, please","I have a special request for my coffee.","Do you have ice coffee?"],
        "responses": ["Certainly, we can customize your coffee as per your preferences.","Sure, we'll make it just the way you like it.","No problem, please let us know your specific requirements."]
    },
    {
        "tag": "Check Availability",
        "patterns": ["Do you have London Fog available?","Is Coffee Grain in stock?","Are you serving London Fog today?","Do you have ice coffee?"],
        "responses": ["Yes, Londong Fog is currently available.","Let me check for you... Yes, London Fog is in stock.","I'm sorry, London Fog is not available today. Would you like something else?"]
    },
    {
        "tag": "Place Repeat Order",
        "patterns": ["I want the same coffee I had last time.","Can you repeat my previous order?","Do you remember what I ordered last time?","The same yesterday, please"],
        "responses": ["Sure, we'll prepare your usual order right away.","Of course, we have your previous order saved. Would you like the same today?","Yes, we have your order history on file. Would you like to order the same as before?"]
    },
    {
        "tag": "Check Order Status",
        "patterns": ["Can you tell me the status of my order?","Has my coffee been prepared yet?","When will my order be ready?","I want to check if my coffee is ready "],
        "responses": ["Let me check on that for you... Your order is currently being prepared.","Your coffee is ready and waiting for pickup!","I'm sorry for the delay, your order will be ready in a few minutes."]
    },
    {
        "tag": "Cancel Order",
        "patterns": ["I need to cancel my order.","Can you cancel my coffee?","I changed my mind, I don't want the coffee anymore.", "Can you cancel my order please?"],
        "responses": ["Sure, your order has been canceled.","No problem, we've canceled your order as requested.","That's alright, your order has been removed from our system."]
    },
    {
        "tag": "Payment Inquiry",
        "patterns": ["What payment methods do you accept?","Do you take credit cards?","Is cash the only payment option?","Can I pay with credit card?"],
        "responses": ["We accept cash, credit/debit cards, and mobile payments.","Yes, we accept all major credit cards.","Cash is one of the payment options, but we also accept cards and digital payments."]
    },
    {
        "tag": "Feedback",
        "patterns": ["I'd like to leave some feedback.","Can I give you some suggestions?","Is there a way to provide feedback about my experience?", "My feedback for this coffee is 10 points."],
        "responses": ["We appreciate your feedback! Please feel free to share.","Absolutely, we welcome any suggestions you may have.","Yes, we value your input. Please share your thoughts with us."]
    },
    {
        "tag": "Order for Pickup",
        "patterns": ["I'd like to pick up my order.","Is my order ready to pick up?","Can I pick up my order in 10 minutes?", "I came to pick up "],
        "responses": ["Sure, your order will be ready for pickup shortly.", "Sure", "Your order will be ready in 2 minutes","Yes, you can pick up in 10 minutes."]
    },
    {
        "tag": "Order for Delivery",
        "patterns": ["Can you deliver my coffee to my address?","Do you offer delivery services?","Are you delivering today?","Are you ok for delivery ?"],
        "responses": ["We offer delivery services. Please provide your address for delivery.","Yes, we provide delivery services. Please provide your address for delivery."]
    },
    {
        "tag": "Special Offers",
        "patterns": ["Do you have any discounts today?","Are there any promotions for regular customers?","Can I use a coupon for my order?", "Is the student discount available?"],
        "responses": ["Yes, we have a special offer on Friday today!","As a regular customer, you qualify for our loyalty program benefits.","Sure, please present your coupon at the checkout for a discount."]
    },
    {
        "tag": "Size Options",
        "patterns": ["What sizes do you offer for coffee?","Can I get my coffee in different sizes?","Do you have small, medium, and large options?", "Can I ask for small size?"],
        "responses": ["Yes, we offer small, medium, and large sizes for our coffee.","You can choose from our range of sizes: small, medium, and large.","Certainly, we have a variety of sizes available to suit your preference."]
    },
    {
        "tag": "Ingredients Inquiry",
        "patterns": ["What ingredients do you use in your coffee?","Are your coffee ingredients organic?","Can you tell me about the milk you use?","Do you use vegan ingredients?"],
        "responses": ["Our coffee is made with freshly ground beans and high-quality milk.","We use organic ingredients in our coffee preparations.","Our milk is sourced locally and is of the highest quality.","Yes we do have vegan option"]
    },
    {
        "tag": "Recommendation Request",
        "patterns": ["Can you recommend a coffee for me?","What's your most popular coffee?","Do you have any specialty coffees you'd recommend?","What is your recommendation?"],
        "responses": ["Our [specialty coffee] is a customer favorite, I highly recommend it!","Our barista's special blend is always a popular choice.","If you're feeling adventurous, I suggest trying our [specialty coffee], it's a unique blend."]
    },
    {
        "tag": "Allergy Information Inquiry",
        "patterns": ["Do you have allergen information for your coffee?","Are there any allergens I should be aware of in your products?","Can you tell me if your coffee contains nuts/dairy/gluten?","Is it gluten free?"],
        "responses": ["We provide allergen information for our coffee products upon request.","Our coffee products are labeled with allergen information for your convenience.","Please let us know if you have any specific allergen concerns, and we'll be happy to assist you.","Yes, it is"]
    }
          
]

In [5]:
# Add new intents to the json file 

hozana_intents.extend(new_intents)

In [6]:
# Store the elements into four lists
tags = []
patterns = []
responses = []
all_intents = []

for intent in hozana_intents:
  for pattern in intent['patterns']:
    tags.append(intent['tag'])
    patterns.append(pattern)
    responses.append(intent['responses'])
    all_intents.append(intent)
    
print(len(tags), len(patterns), len(responses),len(all_intents))

# Displaying the extracted data
print("Tags:", tags)
print("Patterns:", patterns)
print("Responses:", responses)
print("All Intents:", all_intents)

70 70 70 70
Tags: ['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'thanks', 'thanks', 'thanks', 'thanks', 'Order Coffee', 'Order Coffee', 'Order Coffee', 'Order Coffee', 'Customize Order', 'Customize Order', 'Customize Order', 'Customize Order', 'Customize Order', 'Check Availability', 'Check Availability', 'Check Availability', 'Check Availability', 'Place Repeat Order', 'Place Repeat Order', 'Place Repeat Order', 'Place Repeat Order', 'Check Order Status', 'Check Order Status', 'Check Order Status', 'Check Order Status', 'Cancel Order', 'Cancel Order', 'Cancel Order', 'Cancel Order', 'Payment Inquiry', 'Payment Inquiry', 'Payment Inquiry', 'Payment Inquiry', 'Feedback', 'Feedback', 'Feedback', 'Feedback', 'Order for Pickup', 'Order for Pickup', 'Order for Pickup', 'Order for Pickup', 'Order for Delivery', 'Order for Delivery', 'Order for Delivery', 'Order for Delivery', 'Special Offers', 'Special Offers', 'Special Offers', 'Special Offers', 'Size Options', 'Size Options'

In [7]:
# Encoding Intents:

label_encoder = LabelEncoder()
encoded_tags = label_encoder.fit_transform(tags)  # Encoding intents into numeric labels
encoded_tags

array([15, 15, 15, 15, 15, 16, 16, 16, 16,  7,  7,  7,  7,  4,  4,  4,  4,
        4,  2,  2,  2,  2, 11, 11, 11, 11,  3,  3,  3,  3,  1,  1,  1,  1,
       10, 10, 10, 10,  5,  5,  5,  5,  9,  9,  9,  9,  8,  8,  8,  8, 14,
       14, 14, 14, 13, 13, 13, 13,  6,  6,  6,  6, 12, 12, 12, 12,  0,  0,
        0,  0], dtype=int64)

In [8]:
# Print out unique encoded tags and their corresponding intents
for label, intent in zip(encoded_tags, tags):
    print(label, intent)


15 greeting
15 greeting
15 greeting
15 greeting
15 greeting
16 thanks
16 thanks
16 thanks
16 thanks
7 Order Coffee
7 Order Coffee
7 Order Coffee
7 Order Coffee
4 Customize Order
4 Customize Order
4 Customize Order
4 Customize Order
4 Customize Order
2 Check Availability
2 Check Availability
2 Check Availability
2 Check Availability
11 Place Repeat Order
11 Place Repeat Order
11 Place Repeat Order
11 Place Repeat Order
3 Check Order Status
3 Check Order Status
3 Check Order Status
3 Check Order Status
1 Cancel Order
1 Cancel Order
1 Cancel Order
1 Cancel Order
10 Payment Inquiry
10 Payment Inquiry
10 Payment Inquiry
10 Payment Inquiry
5 Feedback
5 Feedback
5 Feedback
5 Feedback
9 Order for Pickup
9 Order for Pickup
9 Order for Pickup
9 Order for Pickup
8 Order for Delivery
8 Order for Delivery
8 Order for Delivery
8 Order for Delivery
14 Special Offers
14 Special Offers
14 Special Offers
14 Special Offers
13 Size Options
13 Size Options
13 Size Options
13 Size Options
6 Ingredients Inqu

In [9]:
tokenizer = Tokenizer(num_words=900)
tokenizer.fit_on_texts(patterns)
sequences = tokenizer.texts_to_sequences(patterns)
padded_sequences = pad_sequences(sequences, maxlen=17)

# Printing some information
print("Encoded Tags:", encoded_tags)
print("Word Index:", tokenizer.word_index)
print("Padded Sequences Shape:", padded_sequences.shape)

Encoded Tags: [15 15 15 15 15 16 16 16 16  7  7  7  7  4  4  4  4  4  2  2  2  2 11 11
 11 11  3  3  3  3  1  1  1  1 10 10 10 10  5  5  5  5  9  9  9  9  8  8
  8  8 14 14 14 14 13 13 13 13  6  6  6  6 12 12 12 12  0  0  0  0]
Word Index: {'you': 1, 'coffee': 2, 'my': 3, 'i': 4, 'can': 5, 'do': 6, 'is': 7, 'for': 8, 'to': 9, 'order': 10, 'the': 11, 'a': 12, 'have': 13, 'are': 14, 'your': 15, 'please': 16, 'want': 17, 'in': 18, 'what': 19, 'there': 20, "i'd": 21, 'like': 22, 'me': 23, 'pick': 24, 'up': 25, 'any': 26, 'use': 27, 'some': 28, 'today': 29, 'tell': 30, 'ready': 31, 'cancel': 32, 'feedback': 33, 'ingredients': 34, 'thanks': 35, 'get': 36, 'with': 37, 'milk': 38, 'ice': 39, 'london': 40, 'fog': 41, 'available': 42, 'same': 43, 'last': 44, 'time': 45, 'of': 46, 'be': 47, 'if': 48, 'payment': 49, 'credit': 50, 'about': 51, '10': 52, 'offer': 53, 'delivery': 54, 'sizes': 55, 'small': 56, 'recommend': 57, 'gluten': 58, 'hi': 59, 'hey': 60, 'anyone': 61, 'hello': 62, 'how': 63, 't

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(padded_sequences, encoded_tags, test_size=0.2) 


In [11]:
max_len = padded_sequences.shape[1]


# Define the deep learning model
model = Sequential([
    Embedding(len(tokenizer.word_index) + 1, 20, input_length=max_len),  # Add +1 for the unknown token
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(10, activation='sigmoid'),
    Dense(len(set(y_train)), activation='softmax')  # Number of classes = total number of unique encoded tags
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# Step 3: Print model summary
print(model.summary())



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 20)            3120      
                                                                 
 global_average_pooling1d (  (None, 20)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 10)                170       
                                                                 
 dense_2 (Dense)             (None, 17)                187       
                                                                 
Total params: 3813 (14.89 KB)
Trainable params: 3813 (14.89 KB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [12]:
model.fit(X_train, y_train, 
          validation_data=(X_val, y_val),
          epochs=500)


Epoch 1/500


2/2 [==============================] - 10s 2s/step - loss: 2.9179 - accuracy: 0.0536 - val_loss: 2.9275 - val_accuracy: 0.0714
Epoch 2/500
2/2 [==============================] - 0s 141ms/step - loss: 2.9129 - accuracy: 0.0536 - val_loss: 2.9268 - val_accuracy: 0.0714
Epoch 3/500
2/2 [==============================] - 0s 125ms/step - loss: 2.9095 - accuracy: 0.0536 - val_loss: 2.9274 - val_accuracy: 0.0714
Epoch 4/500
2/2 [==============================] - 0s 121ms/step - loss: 2.9054 - accuracy: 0.0536 - val_loss: 2.9275 - val_accuracy: 0.0714
Epoch 5/500
2/2 [==============================] - 0s 120ms/step - loss: 2.9022 - accuracy: 0.0536 - val_loss: 2.9276 - val_accuracy: 0.0714
Epoch 6/500
2/2 [==============================] - 0s 121ms/step - loss: 2.8987 - accuracy: 0.0536 - val_loss: 2.9276 - val_accuracy: 0.0714
Epoch 7/500
2/2 [==============================] - 0s 109ms/step - loss: 2.8958 - accuracy: 0.0536 - val_loss: 2.9277 - val_accuracy: 0.0714
Epoch 8/500
2

In [13]:
model.fit(X_train, y_train, 
          validation_data=(X_val, y_val),
          epochs=1000)

Epoch 1/1000
2/2 [==============================] - 1s 556ms/step - loss: 1.3290 - accuracy: 0.6250 - val_loss: 2.9818 - val_accuracy: 0.0714
Epoch 2/1000
2/2 [==============================] - 0s 197ms/step - loss: 1.3268 - accuracy: 0.6250 - val_loss: 2.9828 - val_accuracy: 0.0714
Epoch 3/1000
2/2 [==============================] - 0s 133ms/step - loss: 1.3247 - accuracy: 0.6250 - val_loss: 2.9835 - val_accuracy: 0.0714
Epoch 4/1000
2/2 [==============================] - 0s 193ms/step - loss: 1.3225 - accuracy: 0.6250 - val_loss: 2.9849 - val_accuracy: 0.0714
Epoch 5/1000
2/2 [==============================] - 0s 156ms/step - loss: 1.3204 - accuracy: 0.6250 - val_loss: 2.9864 - val_accuracy: 0.0714
Epoch 6/1000
2/2 [==============================] - 0s 95ms/step - loss: 1.3182 - accuracy: 0.6250 - val_loss: 2.9873 - val_accuracy: 0.0714
Epoch 7/1000
2/2 [==============================] - 0s 81ms/step - loss: 1.3160 - accuracy: 0.6250 - val_loss: 2.9885 - val_accuracy: 0.0714
Epoch 8/

In [14]:

# Load tokenizer from disk
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Load encoder from disk
with open('encoder.pickle', 'rb') as handle:
    label_encoder = pickle.load(handle)

# Load model from disk
model = tf.keras.models.load_model('chatbot_model')

In [16]:
while True:
    # Receive input from the user
    user_input = input("You: ")

    # Tokenize the input
    input_sequence = tokenizer.texts_to_sequences([user_input])
    padded_input_sequence = pad_sequences(input_sequence, maxlen=max_len)

    # Pass the input to the loaded model
    predictions = model.predict(padded_input_sequence)

    # Decode the result
    predicted_intent = label_encoder.inverse_transform([tf.argmax(predictions, axis=1).numpy()[0]])[0]

    # Check if the result matches any of the intents
    for intent in all_intents:
        if intent['tag'] == predicted_intent:
            # Randomly choose a response
            response = random.choice(intent['responses'])
            print("Bot:", response)
            break

    # Check if the user wants to end the conversation
    if user_input.lower() == 'bye':
        print("Bot: Goodbye!")
        break


1/1 [==============================] - 0s 82ms/step
Bot: Great choice! Would you like anything else with your coffee?
1/1 [==============================] - 0s 122ms/step
Bot: Certainly, we can customize your coffee as per your preferences.
1/1 [==============================] - 0s 79ms/step
Bot: Your coffee is ready and waiting for pickup!
1/1 [==============================] - 0s 92ms/step
Bot: Yes, we accept all major credit cards.
1/1 [==============================] - 1s 517ms/step
Bot: We accept cash, credit/debit cards, and mobile payments.
Bot: Goodbye!
